In [3]:
import pandas as pd 
from subs.energy_api import *


df_carbon_forecast_indexed =carbon_api_forecast()


In [4]:
df_carbon_forecast_indexed

,FieldName,Region,Value
EffectiveTime,,,
2024-01-31 16:30:00,CO2_INTENSITY_FORECAST,ALL,204.8873
2024-01-31 17:00:00,CO2_INTENSITY_FORECAST,ALL,207.7346
2024-01-31 17:30:00,CO2_INTENSITY_FORECAST,ALL,213.5658
2024-01-31 18:00:00,CO2_INTENSITY_FORECAST,ALL,211.1651
2024-01-31 18:30:00,CO2_INTENSITY_FORECAST,ALL,214.0227
2024-01-31 19:00:00,CO2_INTENSITY_FORECAST,ALL,215.0443
2024-01-31 19:30:00,CO2_INTENSITY_FORECAST,ALL,218.4071
2024-01-31 20:00:00,CO2_INTENSITY_FORECAST,ALL,227.2394
2024-01-31 20:30:00,CO2_INTENSITY_FORECAST,ALL,239.7786


In [5]:
co2_stats_prior_day, df_carbon_intensity_recent = carbon_api_intensity()

/Users/saeed/Documents/GitHub/telegram-energy-api/subs/energy_api.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_carbon_intensity_recent["Value"] = df_carbon_intensity_recent[


In [12]:
df_carbon_intensity_recent.loc['2024-01-30']

,FieldName,Region,Value
EffectiveTime,,,
2024-01-30 16:45:00,CO2_INTENSITY,ALL,226.0
2024-01-30 17:00:00,CO2_INTENSITY,ALL,219.0
2024-01-30 17:15:00,CO2_INTENSITY,ALL,216.0
2024-01-30 17:30:00,CO2_INTENSITY,ALL,212.0
2024-01-30 17:45:00,CO2_INTENSITY,ALL,209.0
2024-01-30 18:00:00,CO2_INTENSITY,ALL,213.0
2024-01-30 18:15:00,CO2_INTENSITY,ALL,214.0
2024-01-30 18:30:00,CO2_INTENSITY,ALL,208.0
2024-01-30 18:45:00,CO2_INTENSITY,ALL,206.0
